# Hypha Demos

## Install client library

In [ ]:
try:
    # For pyodide in the browser
    import micropip
    await micropip.install(['imjoy_rpc', 'numpy'])
except ImportError:
    # For native python with pip
    import subprocess
    subprocess.call(['pip', 'install', 'imjoy_rpc', 'numpy'])

## Connect to the server

In [ ]:
from imjoy_rpc.hypha import login, connect_to_server

SERVER_URL = "https://ai.imjoy.io"
token = await login({"server_url": SERVER_URL})

server = await connect_to_server(
    {"name": "test client", "server_url": SERVER_URL, "token": token}
)

In [ ]:
# Generate credentials for later usage
workspace = server.config.workspace
token = await server.generate_token()

## List all the public services

In [ ]:
public_services = await server.list_services("public")
public_services

## Use the s3-storage service

In [ ]:
s3 = await server.get_service("s3-storage")
s3

## Generate credential for s3 client

In [ ]:
info = await s3.generate_credential()
info

## Upload an file to the workspace via hypha

In [ ]:
from js import fetch, File, Object, eval, JSON
url = f"{SERVER_URL}/{workspace}/files/hello.txt"
headers = eval(f"headers={{Authorization: 'Bearer {token}'}}")
file = File.new(["hi!"], "hello.txt");
response = await fetch(url, method="PUT", body=file, withCredentials=True, headers=headers)
await response.text()

## Generate a presigned url for downloading a file

In [ ]:
url = await s3.generate_presigned_url(
    info["bucket"], info["prefix"] + "hello.txt"
)
print(url)
response = await fetch(url, method="GET")
await response.text()

## Download the file

In [ ]:
from js import fetch, File, Object, eval
url = f"{SERVER_URL}/{workspace}/files/hello.txt"
headers = eval(f"headers={{Authorization: 'Bearer {token}'}}")
response = await fetch(url, method="GET", withCredentials=True, headers=headers)
await response.text()

In [ ]:
await s3.list_files()

## Use triton-client service

In [ ]:
from js import fetch
import io
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

async def fetch_image(url, name=None, grayscale=False, transpose=False, size=None):
    response = await fetch(url)
    bytes = await response.arrayBuffer()
    bytes = bytes.to_py()
    buffer = io.BytesIO(bytes)
    buffer.name = name or url.split('?')[0].split('/')[1]
    image = Image.open(buffer)
    if grayscale:
        image = image.convert('L')
    if size:
        image = image.resize(size=size)
    image = np.array(image).astype('float32')
    if transpose:
        image = image.transpose(2, 0, 1)
    return image

def display_image(image, mask):
    # display the output
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
    ax1.imshow(image)
    ax1.set_title('input image')
    ax2.imshow(mask)
    ax2.set_title('predicted mask')
    plt.show()

In [ ]:
image = await fetch_image('https://static.imjoy.io/img/img02.png', transpose=True)

In [ ]:
triton = await server.get_service("triton-client")
results = await triton.execute(inputs=[image, {'diameter': 30}], model_name='cellpose-python', decode_json=True)

mask = results['mask'][0]
display_image(image.transpose(1, 2, 0).astype('uint8'), mask)

## Use server-apps service

In [ ]:
controller = await server.get_service("server-apps")
controller

## Launch a window plugin

In [ ]:
TEST_APP_CODE = """
api.log('awesome!connected!');

api.export({
    async setup(){
        console.log("this is a log");
        console.error("this is an error");
        await api.log("initialized")
    },
    async check_webgpu(){
        if ("gpu" in navigator) {
            // WebGPU is supported!
            return true
        }
        else return false
    },
    async execute(a, b){
        console.log("executing", a, b);
        return a + b
    }
})
"""
controller = await server.get_service("server-apps")
config = await controller.launch(
    source=TEST_APP_CODE,
    config={"type": "window"},
)
assert "app_id" in config
plugin = await server.get_plugin(config.id)
assert "execute" in plugin
result = await plugin.execute(2, 4)
assert result == 6
webgpu_available = await plugin.check_webgpu()
assert webgpu_available is True
# only pass name so the app won't be removed
await controller.stop(config.id)

## Launch a web-python plugin

In [ ]:
TEST_APP_CODE = """
from imjoy_rpc import api

async def setup():
    await api.log("started")

api.export({"setup": setup})
"""
controller = await server.get_service("server-apps")
config = await controller.launch(
    source=TEST_APP_CODE,
    config={"type": "web-python"},
)
plugin = await server.get_plugin(config.id)
assert "setup" in plugin

In [ ]:
await plugin.setup()

In [ ]:
plugins = await server.list_plugins()
plugins

In [ ]:
await controller.get_log(config.id)

In [ ]:
await controller.stop(config.id)

## Launch a web-python plugin and serve a web app

In [ ]:
controller = await server.get_service("server-apps")
app = await controller.launch(
    source="https://gist.githubusercontent.com/oeway/4667231efb08c39b9f41b827b91229de/raw/ASGIWebPythonPlugin.imjoy.html",
    timeout=20
)
server_info = await server.get_connection_info()

print(f"Server app running at {server_info.public_base_url}/{workspace}/apps/cellpose/predict")

In [ ]:
app

In [ ]:
await controller.get_log(app.id)

In [ ]:
apps = await controller.list_running()
for app in apps:
    print("stopping " + app.id)
    await controller.stop(app.id)